# Phindr3D in python

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial import distance as dist
import pandas as pd
import phindr_functions as phi
import metadata_functions as mf

# Create Metadata
Uncomment the code below to create metadata if needed. Regex and paths must be edited to reflect desired images and directories.

In [ ]:
# folder_path = r'C:\Users\teole\anaconda3\envs\phy479\phindrtestsynthetic'
# regex = 'image(?P<synthImID>\d+)__(?P<Treatment>.+)__z(?P<Stack>\d+)__ch(?P<Channel>\d)__dla.tiff'

# mdata_path = mf.createMetadata(folder_path, regex)

# Preliminary set-up:

In [ ]:
param = phi.initParameters()

# EDIT HERE
#change parameters as needed
#Super-Voxel dimensions
param.tileX = 10
param.tileY = 10
param.tileZ = 3
#Mega-Voxel dimensions
param.megaVoxelTileX = 5
param.megaVoxelTileY = 5
param.megaVoxelTileZ = 2
#Number of categories for binning
param.numVoxelBins = 20
param.numSuperVoxelBins = 15
param.numMegaVoxelBins = 40
#foreground thresholds
param.intensityThresholdTuningFactor = 0.5
param.superVoxelThresholdTuningFactor = 0.5
param.megaVoxelThresholdTuningFactor = 0.5
#training parameters:
param.randTrainingSuperVoxel = 10000
param.pixelsPerImage = 200
param.randTrainingFields = 5
param.randTrainingPerTreatment = 1
param.intensityNormPerTreatment = False
#visualization:
param.showBincenters = False
param.showImage = False #may be more useful to change this lower down, otherwise may show large number of images
param.showChannels = False # show individual channels instead of rgb images.

#include texture features of Mega Voxel image:
param.textureFeatures = True

#misc. (generally don't need to touch)
param.minQuantileScaling = .5
param.maxQuantileScaling = .5
param.countBackground = False
param.showImage = 0
param.startZPlane = 1
param.endZPlane = 500
param.numRemoveZStart = 1
param.numRemoveZEnd = 1
param.computeTAS = 0
param.trainingPerColumn = False
param.treatmentCol = ''
param.imageTreatments = []
param.allTreatments = []
param.trainingColforImageCategories = []
param.superVoxelPerField = param.randTrainingSuperVoxel//param.randTrainingFields
param.lowerbound = [0, 0, 0]
param.upperbound = [1, 1, 1]
param.numChannels = 3 

#don't need to touch these 
param.svcolormap = phi.random_cmap(map_len=param.numSuperVoxelBins+1)
param.mvcolormap = phi.random_cmap(map_len=param.numMegaVoxelBins+1)

param.showImage = False #################################shouldnt rlly be here.

# File loading:
Load folder containing images of interest

NOTE:

If the `analysis_folder_path` is different from the `training_folder_path`, then the Profile Images section requires editing to ensure the images for analysis are loaded properly.

In [ ]:
# metadatafile based stuff here.

mdata_path = r'E:\Phindr3D_organoid-sample-data\guiSegmentation\SegmentedImages\metadata_python.txt'
mdata = pd.read_csv(mdata_path, sep='\t')
header = mdata.columns
channels = header[header.map(lambda head: head.startswith('Channel_'))]
imageID = header[header == 'ImageID']
filenameData = header[header.map(lambda head: (head not in channels) and (head not in imageID) and (head != 'MetadataFile') and (head != 'Stack'))]
stack = header[header == 'Stack']
uImageID = np.unique(mdata[imageID])
treatment = header[header == 'Treatment']

param.numChannels = len(channels)
param.treatmentCol = treatment
param.channelCol = channels
param.imageIDCol = imageID
param.stackCol = stack
param.allTreatments = np.unique(mdata[param.treatmentCol])

mdata



# Calculate image parameters:

Get image scaling factors and intensity thresholds.

In [ ]:
param = phi.getScalingFactorforImages(mdata, param)

print('Lowerbounds:', param.lowerbound)
print('Upperbounds:', param.upperbound)

param = phi.getImageThresholdValues(mdata, param)

intensityThreshold = np.quantile(param.intensityThresholdValues, param.intensityThresholdTuningFactor, axis=0)
param.intensityThreshold = np.reshape(intensityThreshold, (1, param.numChannels))

print('Intensity thresholds:')
print(param.intensityThreshold)

# Cluster pixels, supervoxels, megavoxels:
compute all categories 

In [ ]:
param = phi.getPixelBinCenters(mdata, param)
param = phi.getSuperVoxelBinCenters(mdata, param)
param = phi.getMegaVoxelBinCenters(mdata, param)

# print(param.pixelBinCenters) #20 different pixel categories made up of scaled intensities of different channels
# print(param.pixelBinCenters.shape)

# Profile images:
Commmented cell can be used to anaysize image sets different from image set.

In [ ]:
# # # # uncomment to load separate analysis metadata file

# mdata_path = r'E:\noclaheoneimage\SegmentedImages\metadata_python.txt'
# mdata = pd.read_csv(mdata_path, sep='\t')
# header = mdata.columns
# channels = header[header.map(lambda head: head.startswith('Channel_'))]
# imageID = header[header == 'ImageID']
# filenameData = header[header.map(lambda head: (head not in channels) and (head not in imageID) and (head != 'MetadataFile') and (head != 'Stack'))]
# stack = header[header == 'Stack']
# uImageID = np.unique(mdata[imageID])
# treatment = header[header == 'Treatment']

# param.numChannels = len(channels)
# param.treatmentCol = treatment
# param.channelCol = channels
# param.imageIDCol = imageID
# param.stackCol = stack
# param.allTreatments = np.unique(mdata[param.treatmentCol])


In [ ]:
output_file_name = r"C:\Users\teole\anaconda3\envs\phy479\PhindrResults\pythontest.txt"

%matplotlib inline
#Set to True to show images in process (will show LARGE number of images): EDIT HERE
param.showImage = True

param, resultIM, resultRaw, df = phi.extractImageLevelTextureFeatures(mdata, param, outputFileName=output_file_name)

from IPython.display import display
display(df)

In [ ]:
# output_file_name = "trainingtest.csv"

# %matplotlib inline
# #Set to True to show images in process (will show LARGE number of images): EDIT HERE
# param.showImage = True

# param, resultIM, resultRaw, df = phi.extractImageLevelTextureFeatures(mdata, param, outputFileName=output_file_name, outputDir='')

# from IPython.display import display
# display(df)